### RAG Basics
##### Boilerplate code

In [ ]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)


In [ ]:
# openai_llm.invoke("Who is Dahaa in coolie movie 2025 tamil?").content
# print(openai_llm.invoke("What is your knowledge cutoff point? 2023?").content)
# print(google_llm.invoke("What is your knowledge cutoff point? 2023?").content)

##### 1. Loading the document

In [ ]:
# Load, split, and embed the documents

from langchain_community.document_loaders import TextLoader

loader = TextLoader("./docs_for_rag/coolie_large.txt")

documents = loader.load()

print(documents)

##### 2. Splitting the documents (also called as chunking)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pprint import pprint

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)


print(len(chunks))
pprint(chunks)

##### 3. Embedding and storing it to vector store

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

chunk_texts = [document.page_content for document in chunks] 
doc_vectors = embeddings.embed_documents(chunk_texts)
print(doc_vectors[0])

##### 4. Storing it to vector database

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

results = vectorstore.similarity_search( #asimilarity_search
    "who is dahaa?"
)
print(results)

##### 5. Making the vectorstore runnable

In [ ]:
retriever = vectorstore.as_retriever() # converts the vectorstore to runnable

print(retriever.invoke("Who is dahaa?"))

##### 5. Feed and get response from LLM

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# Create RAG chain using LCEL
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt_template = ChatPromptTemplate.from_template("""
        Use the following context to answer the question:
        Context: {context}
        Question: {question}

        Answer:""")


chain = {"context": retriever, "question": RunnablePassthrough()} | prompt_template | google_llm | StrOutputParser()

print(chain.invoke("Who is Dahaa in coolie movie 2025 tamil?"))
# print(chain.invoke("தாஹா என்பவன் யார்?"))
# print(chain.invoke("Which file holds this info?"))